<a href="https://colab.research.google.com/github/LifeGains/AI_projects/blob/main/Best_Restaurants_Finder_(Google_Maps_API).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
pip install googlemaps

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40711 sha256=6e04e0f01a05f74ea6ff4cce1ecafd6cc1c4c0848feab826d208b3b7efc57eda
  Stored in directory: /root/.cache/pip/wheels/17/f8/79/999d5d37118fd35d7219ef57933eb9d09886c4c4503a800f84
Successfully built googlemaps


In [2]:
pip install ppprint

In [3]:
import os
import googlemaps
from ppprint import ppprint
import pandas as pd
import numpy as np
import time

pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)

In [4]:
# mount google drive
# ---------------------------------
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
# Yes you need to cd there even with the line below.
%cd /content/drive/My\ Drive/Github/secret-api-keys

/content/drive/My Drive/Github/secret-api-keys


In [21]:
# import secret.py
import sys
import os
py_file_location = "/content/drive/My\ Drive/Github/secret-api-keys"
sys.path.append(os.path.abspath(py_file_location))
import secret

# Find Best Restaurants

- Major Source: https://googlemaps.github.io/google-maps-services-python/docs/

- Master list of Place types: https://developers.google.com/maps/documentation/places/web-service/supported_types

In [24]:
gmaps = googlemaps.Client(secret.apikey)

In [25]:
# dir(map_client)

Source: https://developers.google.com/maps/documentation/javascript/places

Nearby Search returns a list of nearby places based on a user's location.

**Text Search returns a list of nearby places based on a search string, eg. "Pizza".**

Place Details requests return more detailed information about a specific place, including user reviews.

In [26]:
import warnings

# Set ignore_warnings to True
warnings.simplefilter("ignore")

In [27]:
def find_best_restaurants(city_name, place_type, min_rating=0, min_n_ratings=0, query='', n_meters=3000):
  lat = gmaps.places(query=city_name).get('results')[0].get('geometry').get('location').get('lat')
  lng = gmaps.places(query=city_name).get('results')[0].get('geometry').get('location').get('lng')
  df = pd.DataFrame()
  min_rating = min_rating
  min_n_ratings = min_n_ratings
  next_page_token_list = ['', '', '']
  # print(len(next_page_token_list))
  counter = 1

  for i in range(len(next_page_token_list)):
    parent_results = gmaps.places(location=(lat,lng),
                                  type = place_type,
                                  query = query,
                                  radius = n_meters,
                                  page_token = next_page_token_list[i],
                                  # minprice = minprice,
                                  # maxprice = maxprice
                                  )
    results = parent_results.get('results')

    for j in range(len(results)):
      if results[j].get('user_ratings_total') >= min_n_ratings and results[j].get('rating') >= min_rating:
        # df = df.concat(results[j], ignore_index=True)
        df = df.append(results[j], ignore_index=True)
    try:
      next_page_token_list[counter] = parent_results.get('next_page_token')
      print(next_page_token_list)
      counter += 1
      time.sleep(3)
    except:
      time.sleep(0)

  df = df.join(pd.json_normalize(df['geometry'])).drop('geometry',axis=1)
  df = df.sort_values(by=['rating', 'user_ratings_total'], ascending=[False, False])
  df = df[ ['name', 'rating', 'user_ratings_total'] + [ col for col in df.columns if col != ['name', 'rating', 'user_ratings_total'] ] ]
  print(len(df))
  return df

In [28]:
# Put the sub-city into the city_name for it to work better. Eg. Lower East Side instead of Manhattan.
find_best_restaurants(city_name='San Jose',
                      # place_type is more set in stone, has to be allowed param in google
                      place_type='restaurant',
                      min_rating=4.3,
                      min_n_ratings=100,
                      # can be park, asian, bar, etc.
                      # query='asian',
                      # radius is in meters, so 4828.02 = 3 miles.
                      n_meters =4828.02,
                      )

# Verify there are 60 results
# find_best_restaurants('morgan hill', 'restaurant')

['', 'AcJnMuEY-6ve_cXjAAGQrFgPJQfKHWsbtwBtQ9w3JRdtTQdWrOqw5Fdgw6NdR2oTz41-haMIKSxCItyG3RtaK31wv1W2WLieTIOrkiVZ7uwHTZ5b2mwHUWRqy3vC8KdN0q6YtB_kEroP6qkFmHJEH_nn7h6T5uwX6FopJvboV_gKq5JDh6Wf7lgc5raoDfozIdgrV0zBfoToP_kY3ig7hTamsxKydb2nJSAhscclhtPZFrFz30SGrqIhH4tdnl9Y5S7Wt1FJggfbrugbQTiFooGByMQos7xXcGIk-e9brNUpg_LevxMwngOm510gY9KiMcZFbrtkHq4MDBI9qDEXUHK8Fg5a_poIAixNhOvtKBnC-7-BzcXxNe7gxp3w6ai4m-K3PH9V8YXd7x05cSiK0JZbeEkXHFc6vhxvbmUborBqZOn97FL9BB6ZVYglIeWfiXfe9OI', '']
['', 'AcJnMuEY-6ve_cXjAAGQrFgPJQfKHWsbtwBtQ9w3JRdtTQdWrOqw5Fdgw6NdR2oTz41-haMIKSxCItyG3RtaK31wv1W2WLieTIOrkiVZ7uwHTZ5b2mwHUWRqy3vC8KdN0q6YtB_kEroP6qkFmHJEH_nn7h6T5uwX6FopJvboV_gKq5JDh6Wf7lgc5raoDfozIdgrV0zBfoToP_kY3ig7hTamsxKydb2nJSAhscclhtPZFrFz30SGrqIhH4tdnl9Y5S7Wt1FJggfbrugbQTiFooGByMQos7xXcGIk-e9brNUpg_LevxMwngOm510gY9KiMcZFbrtkHq4MDBI9qDEXUHK8Fg5a_poIAixNhOvtKBnC-7-BzcXxNe7gxp3w6ai4m-K3PH9V8YXd7x05cSiK0JZbeEkXHFc6vhxvbmUborBqZOn97FL9BB6ZVYglIeWfiXfe9OI', 'AcJnMuFr22HFgVDshWE5N2YKVp1dnzCYwHTD2VhD-RxXJgcrQuPfeRT4EN8Vbe8YE6Y

,name,rating,user_ratings_total,business_status,formatted_address,icon,icon_background_color,icon_mask_base_uri,name,opening_hours,photos,place_id,plus_code,price_level,rating,reference,types,user_ratings_total,location.lat,location.lng,viewport.northeast.lat,viewport.northeast.lng,viewport.southwest.lat,viewport.southwest.lng
5,MAR Y SOL RESTAURANT,4.70,149,OPERATIONAL,"1179 E Santa Clara St, San Jose, CA 95116",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,MAR Y SOL RESTAURANT,{'open_now': True},"[{'height': 2735, 'html_attributions': ['<a hr...",ChIJi7ZXRevNj4AR1SPueCQKGrs,"{'compound_code': '84WM+X2 San Jose, Californi...",2,4.70,ChIJi7ZXRevNj4AR1SPueCQKGrs,"[restaurant, food, point_of_interest, establis...",149,37.35,-121.87,37.35,-121.87,37.35,-121.87
22,Green Lotus Restaurant,4.60,333,OPERATIONAL,"1143 Story Rd #180, San Jose, CA 95122",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Green Lotus Restaurant,{'open_now': True},"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJ_9rz9irNj4ARwiuzsURtQbM,"{'compound_code': '84MW+47 San Jose, Californi...",2,4.60,ChIJ_9rz9irNj4ARwiuzsURtQbM,"[restaurant, point_of_interest, food, establis...",333,37.33,-121.85,37.33,-121.85,37.33,-121.86
15,Mudai Ethiopian Restaurant,4.60,182,OPERATIONAL,"503 W San Carlos St, San Jose, CA 95126",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Mudai Ethiopian Restaurant,{'open_now': True},"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJD19wtDHNj4ARsjNXf_k8hG8,"{'compound_code': '84G2+8F San Jose, Californi...",2,4.60,ChIJD19wtDHNj4ARsjNXf_k8hG8,"[restaurant, food, point_of_interest, establis...",182,37.33,-121.90,37.33,-121.90,37.32,-121.90
21,The Boiling Crab,4.50,2354,OPERATIONAL,"71 Curtner Ave Suite 20, San Jose, CA 95125",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,The Boiling Crab,{'open_now': True},"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJmWZMuxszjoAR0fXhXB1XNV4,"{'compound_code': '843P+38 San Jose, Californi...",2,4.50,ChIJmWZMuxszjoAR0fXhXB1XNV4,"[restaurant, point_of_interest, food, establis...",2354,37.30,-121.86,37.30,-121.86,37.30,-121.87
1,Palermo Italian Restaurant,4.50,1116,OPERATIONAL,"791 Auzerais Ave, San Jose, CA 95126",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Palermo Italian Restaurant,{'open_now': True},"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJw40UG1bLj4ARpK_t0h8GWsI,"{'compound_code': '83CX+P4 San Jose, Californi...",2,4.50,ChIJw40UG1bLj4ARpK_t0h8GWsI,"[bar, restaurant, food, point_of_interest, est...",1116,37.32,-121.90,37.32,-121.90,37.32,-121.90
20,Minato Japanese Restaurant,4.50,649,OPERATIONAL,"617 N 6th St, San Jose, CA 95112",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Minato Japanese Restaurant,{'open_now': True},"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJv6Lt-4TMj4ARLZgLz0LCB2g,"{'compound_code': '84X4+WG San Jose, Californi...",2,4.50,ChIJv6Lt-4TMj4ARLZgLz0LCB2g,"[restaurant, point_of_interest, food, establis...",649,37.35,-121.89,37.35,-121.89,37.35,-121.89
8,Maza Restaurant,4.50,413,OPERATIONAL,"782 N 13th St, San Jose, CA 95112",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Maza Restaurant,{'open_now': True},"[{'height': 2252, 'html_attributions': ['<a hr...",ChIJuYh5XojMj4ARNxWDZWNPswU,"{'compound_code': '9446+MG San Jose, Californi...",1,4.50,ChIJuYh5XojMj4ARNxWDZWNPswU,"[restaurant, food, point_of_interest, establis...",413,37.36,-121.89,37.36,-121.89,37.36,-121.89
11,Gojo Ethiopian Restaurant,4.50,294,OPERATIONAL,"1261 W San Carlos St, San Jose, CA 95126",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://

# v2

In [29]:
# # Define search parameters
# city_name = 'North San Jose'

# lat = gmaps.places(query=city_name).get('results')[0].get('geometry').get('location').get('lat')
# lng = gmaps.places(query=city_name).get('results')[0].get('geometry').get('location').get('lng')

# search_params = {
#     'location': str(lat)+','+str(lng),  # Define the location you want to search near
#     'radius': 1000,                   # Radius in meters
#     'keyword': 'restaurant',          # Keyword for the search (e.g., 'restaurant')
#     'minprice': 0,                   # Minimum price level
#     'maxprice': 4,                   # Maximum price level
# }

# # Perform the search
# places = gmaps.places_nearby(**search_params)

# # Filter and sort results
# filtered_places = [place for place in places['results'] if place['user_ratings_total'] >= 100 and place['rating'] >= 4.1]
# sorted_places = sorted(filtered_places, key=lambda x: x['name'])  # Sort by restaurant name

# # Display the top 10 results
# top_10_places = sorted_places[:10]
# for place in top_10_places:
#     print(f"Name: {place['name']}, Rating: {place['rating']}, Total Reviews: {place['user_ratings_total']}")
